# Approximate q-learning

In this notebook you will teach a __tensorflow__ neural network to do Q-learning.

__Frameworks__ - we'll accept this homework in any deep learning framework. This particular notebook was designed for tensorflow, but you will find it easy to adapt it to almost any python-based deep learning framework.

In [ ]:
#XVFB will be launched if you run on a server
import os
if os.environ.get("DISPLAY") is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

In [ ]:
import gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
env = gym.make("CartPole-v0").env
env.reset()
n_actions = env.action_space.n
state_dim = env.observation_space.shape

plt.imshow(env.render("rgb_array"))

# Approximate (deep) Q-learning: building the network

To train a neural network policy one must have a neural network policy. Let's build it.


Since we're working with a pre-extracted features (cart positions, angles and velocities), we don't need a complicated network yet. In fact, let's build something like this for starters:

![img](https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/yet_another_week/_resource/qlearning_scheme.png)

For your first run, please only use linear layers (L.Dense) and activations. Stuff like batch normalization or dropout may ruin everything if used haphazardly. 

Also please avoid using nonlinearities like sigmoid & tanh: agent's observations are not normalized so sigmoids may become saturated from init.

Ideally you should start small with maybe 1-2 hidden layers with < 200 neurons and then increase network size if agent doesn't beat the target score.

In [ ]:
import tensorflow as tf
import keras
import keras.layers as L
tf.reset_default_graph()
sess = tf.InteractiveSession()
keras.backend.set_session(sess)

In [ ]:
print('We are working with a state dim of {}, and an action space of {}'.format(state_dim, n_actions))

In [ ]:
# NOTE: Suggestion from others: Network trains better with more nodes
nodes = 300

In [ ]:
network = keras.models.Sequential()
network.add(L.InputLayer(state_dim))
network.add(L.ReLU())
network.add(L.Dense(nodes))
network.add(L.ReLU())
network.add(L.Dense(nodes))
network.add(L.ReLU())
network.add(L.Dense(n_actions))

In [ ]:
def get_action(state, epsilon=0):
    """
    Sample actions with epsilon-greedy policy
    
    With p = epsilon pick random action, else pick action with highest Q(s,a)
    
    Parameters
    ----------
    state : np.array, shape (4,)
        The input state vector
    epsilon : float
        The probability of chosing a random action
    
    Returns
    -------
    action : int
        The integer representing the action
    """
    
    # This returns a 2-d array (one q value for each action)
    q_values = network.predict(state[None])[0]
    n_actions = len(q_values)
    
    best_q_value = np.argmax(q_values)
    
    possibilities = [best_q_value] + list(range(n_actions))
    probabilities = [1-epsilon] + [epsilon/n_actions] * n_actions
    
    action = np.random.choice(possibilities, p=probabilities)
    
    return action

In [ ]:
assert network.output_shape == (None, n_actions), "please make sure your model maps state s -> [Q(s,a0), ..., Q(s, a_last)]"
assert network.layers[-1].activation == keras.activations.linear, "please make sure you predict q-values without nonlinearity"

# test epsilon-greedy exploration
s = env.reset()
assert np.shape(get_action(s)) == (), "please return just one action (integer)"
for eps in [0., 0.1, 0.5, 1.0]:
    state_frequencies = np.bincount([get_action(s, epsilon=eps) for i in range(10000)], minlength=n_actions)
    best_action = state_frequencies.argmax()
    assert abs(state_frequencies[best_action] - 10000 * (1 - eps + eps / n_actions)) < 200
    for other_action in range(n_actions):
        if other_action != best_action:
            assert abs(state_frequencies[other_action] - 10000 * (eps / n_actions)) < 200
    print('e=%.1f tests passed'%eps)

### Q-learning via gradient descent

We shall now train our agent's Q-function by minimizing the TD loss:
$$ L = { 1 \over N} \sum_i (Q_{\theta}(s,a) - [r(s,a) + \gamma \cdot max_{a'} Q_{-}(s', a')]) ^2 $$


Where
* $s, a, r, s'$ are current state, action, reward and next state respectively
* $\gamma$ is a discount factor defined two cells above.

The tricky part is with  $Q_{-}(s',a')$. From an engineering standpoint, it's the same as $Q_{\theta}$ - the output of your neural network policy. However, when doing gradient descent, __we won't propagate gradients through it__ to make training more stable (see lectures).

To do so, we shall use `tf.stop_gradient` function which basically says "consider this thing constant when doingbackprop".

In [ ]:
# Create placeholders for the <s, a, r, s'> tuple and a special indicator for game end (is_done = True)
# NOTE: state_dim is a tuple, and can thus be concatenated with another tuple
states_ph = keras.backend.placeholder(dtype='float32', shape=(None,) + state_dim)
actions_ph = keras.backend.placeholder(dtype='int32', shape=[None])
rewards_ph = keras.backend.placeholder(dtype='float32', shape=[None])
next_states_ph = keras.backend.placeholder(dtype='float32', shape=(None,) + state_dim)
is_done_ph = keras.backend.placeholder(dtype='bool', shape=[None])

In [ ]:
# Get q-values for all actions in current states
# NOTE: network(states_ph) inserts the states to the network and outputs the prediction
#       Recall: One input example equals one row (where the columns are features)
#               The corresponding output is at the same row (and the columns are the output classes)
predicted_qvalues = network(states_ph)

# Select q-values for chosen actions
# NOTE: reduce_sum takes the sum, axis=1 makes the sum over rows
#       See "Own notes" below for explaination
predicted_qvalues_for_actions = tf.reduce_sum(predicted_qvalues * tf.one_hot(actions_ph, n_actions), axis=1)

### Own notes

In [ ]:
s = [[0.1, 0.1, 0.1, 0.1], [0, 0, 0, 0], [0.2, 0.2, 0.2, 0.2]]
sess.run(network(states_ph), {states_ph: s})

In [ ]:
# NOTE: The one_hot has a "on_value" of 1
#       The input is the index which should be "on" in the one-hot encoding
a = [1, 2, 0]
sess.run(tf.one_hot(actions_ph, n_actions), {actions_ph: a})

In [ ]:
# NOTE: The one-hot is selecting the q-value of the selected state
#       one_hot takes the input indices, depth
sess.run(predicted_qvalues * tf.one_hot(actions_ph, n_actions), {states_ph: s, actions_ph: a})

In [ ]:
# NOTE: The reduce sum casts the selected values to an array
sess.run(predicted_qvalues_for_actions, {states_ph: s, actions_ph: a})

### Continuation of assignment

In [ ]:
gamma = 0.99

# Compute q-values for all actions in next states
predicted_next_qvalues = network(next_states_ph)

# Compute V*(next_states) using predicted next q-values
next_state_values = tf.reduce_max(predicted_next_qvalues, axis=1)

# Compute "target q-values" for loss - it's what's inside square parentheses in the above formula.
target_qvalues_for_actions = rewards_ph + gamma*next_state_values

# At the last state we shall use simplified formula: Q(s,a) = r(s,a) since s' doesn't exist
# NOTE: If is_done_ph is 1 (True), the rewards are returned, else the target_qvalues_for_actions is returned
target_qvalues_for_actions = tf.where(is_done_ph, rewards_ph, target_qvalues_for_actions)

### Own notes

In [ ]:
# NOTE: We are reusing s as the next state (I'm too lazy to make a new array)
sess.run(predicted_next_qvalues, {next_states_ph: s})

In [ ]:
# NOTE: The reduce_max operation did the same as the complicated one_hot in predicted_qvalues_for_actions
#       The middle is here as 2 returns a 0-array for one_hot with depth = 2
sess.run(next_state_values, {next_states_ph: s})

### Continuation of assignment

In [ ]:
# Mean squared error loss to minimize
loss = (predicted_qvalues_for_actions - tf.stop_gradient(target_qvalues_for_actions)) ** 2
loss = tf.reduce_mean(loss)

# training function that resembles agent.update(state, action, reward, next_state) from tabular agent
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [ ]:
assert tf.gradients(loss, [predicted_qvalues_for_actions])[0] is not None, "make sure you update q-values for chosen actions and not just all actions"
assert tf.gradients(loss, [predicted_next_qvalues])[0] is None, "make sure you don't propagate gradient w.r.t. Q_(s',a')"
assert predicted_next_qvalues.shape.ndims == 2, "make sure you predicted q-values for all actions in next state"
assert next_state_values.shape.ndims == 1, "make sure you computed V(s') as maximum over just the actions axis and not all axes"
assert target_qvalues_for_actions.shape.ndims == 1, "there's something wrong with target q-values, they must be a vector"

### Playing the game

In [ ]:
def generate_session(t_max=1000, epsilon=0, train=False):
    """
    Play env with approximate q-learning agent and train it at the same time
    
    Parameters
    ----------
    t_max : int
        Maximum steps to play before stop (if not stopped by ending the game)
    epsilon : float
        The probability of taking an random action
    train : bool
        Whether or not to train
        
    Returns
    -------
    total_reward : float
        The total reward
    """
    
    total_reward = 0
    s = env.reset()
    
    for t in range(t_max):
        a = get_action(s, epsilon=epsilon)       
        next_s, r, done, _ = env.step(a)
        
        if train:
            sess.run(train_step,{
                states_ph: [s], actions_ph: [a], rewards_ph: [r], 
                next_states_ph: [next_s], is_done_ph: [done]
            })

        total_reward += r
        s = next_s
        if done: 
            break
            
    return total_reward

In [ ]:
epsilon = 0.5

In [ ]:
for i in range(1000):
    session_rewards = [generate_session(epsilon=epsilon, train=True) for _ in range(100)]
    print("epoch #{}\tmean reward = {:.3f}\tepsilon = {:.3f}".format(i, np.mean(session_rewards), epsilon))
    
    epsilon *= 0.99
    assert epsilon >= 1e-4, "Make sure epsilon is always nonzero during training"
    
    # NOTE: We train a bit longer to ensure that we succeed in the submission
    if np.mean(session_rewards) > 400:
        print ("You Win!")
        break

In [ ]:
saver = tf.train.Saver()
save_path = saver.save(sess, './model.ckpt')

### How to interpret results


Welcome to the f.. world of deep f...n reinforcement learning. Don't expect agent's reward to smoothly go up. Hope for it to go increase eventually. If it deems you worthy.

Seriously though,
* __mean reward__ is the average reward per game. For a correct implementation it may stay low for some 10 epochs, then start growing while oscilating insanely and converges by ~50-100 steps depending on the network architecture. 
* If it never reaches target score by the end of for loop, try increasing the number of hidden neurons or look at the epsilon.
* __epsilon__ - agent's willingness to explore. If you see that agent's already at < 0.01 epsilon before it's is at least 200, just reset it back to 0.1 - 0.5.

### Record videos

As usual, we now use `gym.wrappers.Monitor` to record a video of our agent playing the game. Unlike our previous attempts with state binarization, this time we expect our agent to act ~~(or fail)~~ more smoothly since there's no more binarization error at play.

As you already did with tabular q-learning, we set epsilon=0 for final evaluation to prevent agent from exploring himself to death.

In [ ]:
env.close()

In [ ]:
#record sessions
import gym.wrappers
env = gym.wrappers.Monitor(gym.make("CartPole-v0"), directory="videos", force=True)
sessions = [generate_session(epsilon=0, train=False) for _ in range(100)]
env.close()

In [ ]:
#show video
from IPython.display import HTML
import os

video_names = list(filter(lambda s:s.endswith(".mp4"),os.listdir("./videos/")))

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format("./videos/"+video_names[-1])) #this may or may not be _last_ video. Try other indices

### Submit to coursera

In [ ]:
EMAIL = ''
TOKEN = ''

In [ ]:
from submit import submit_cartpole
submit_cartpole(generate_session, EMAIL, TOKEN)